In [49]:
import pandas as pd
import numpy as np

In [50]:
london_flows=pd.read_csv("data/london_flows.csv")
london_flows=london_flows[london_flows['station_origin']!=london_flows["station_destination"]]
london_flows=london_flows[london_flows['distance'] != 0]# drop intra station
london_flows=london_flows[london_flows['station_origin'] != 'Battersea Park']
london_flows=london_flows[london_flows['station_destination'] != 'Battersea Park']


In [51]:
# change column name
london_flows=london_flows.rename(columns={"station_origin":"Orig","station_destination":'Dest','flows':'Total','distance':'Dist','population':'Oi_origpop','jobs':'Dj_destjob'})

In [52]:
# create a pivot table to monitor flows
ldf_submat=pd.pivot_table(london_flows,values ="Total", index="Orig", columns = "Dest",
                            aggfunc=np.sum, margins=True)


In [53]:
ldf_submat

Dest,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
Orig,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,599
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1224
Acton Town,NaN,NaN,NaN,3.0,17.0,NaN,35.0,0.0,NaN,11.0,...,77.0,3.0,6.0,9.0,NaN,0.0,NaN,0.0,NaN,3745
Aldgate,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,17.0,...,0.0,NaN,4.0,8.0,NaN,0.0,NaN,0.0,NaN,2886
Aldgate East,NaN,NaN,2.0,0.0,NaN,NaN,0.0,0.0,NaN,20.0,...,24.0,0.0,0.0,12.0,NaN,1.0,NaN,1.0,NaN,3172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,5.0,47.0,NaN,NaN,NaN,NaN,22.0,...,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4868
Woodgrange Park,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530
Woodside Park,NaN,NaN,1.0,26.0,11.0,NaN,0.0,NaN,NaN,59.0,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3093


## Evalutation of model
Goodness of fit
*coefficient of determination ($r^2$)
 $r^2$ is popular as it is quite intuitive and can be compared across models.
*the Square Root of Mean Squared Error (RMSE).
Less intuitive but can be used to comparing changes to the same model.


In [54]:
import scipy.stats

def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2


In [55]:
from math import sqrt

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE


In [56]:
# take log of data
import statsmodels.api as sm
import statsmodels.formula.api as smf

#take the variables and produce logarithms of them
x_variables = ["Oi_origpop", "Dj_destjob", "Dist"]
log_x_vars = []
for x in x_variables:
    london_flows[f"log_{x}"] = np.log(london_flows[x])
    log_x_vars.append(f"log_{x}")


In [ ]:
london_flows.head()

## 4. Doubly Constrained Model

For the doubly constrained model:

- We might be interested in understanding the changing patterns of commuting or migration over time. Data from the Census allows us to know an accurate snap-shot of migrating and commuting patterns every 10 years. In these full data matrices, we know both the numbers of commuters/migrants leaving origins and arriving at destinations as well as the interactions between them. If we constrain our model estimates to this known information at origin and destination, we can examine various things, including:
    - The ways that the patterns of commuting/migration differ from the model predictions - where we might get more migrant/commuter flows than we would expect
    - How the model parameters vary over time - for example how does distance / cost of travel affect flows over time? Are people prepared to travel further or less far than before?


Which we can now introduce.

Let us begin with the formula:

\begin{equation} \tag{9}
T_{ij} = A_i B_j O_i D_j d_{ij}^{-\beta}
\end{equation}

Where

\begin{equation} \tag{10}
O_i = \sum_j T_{ij}
\end{equation}

\begin{equation} \tag{11}
D_j = \sum_i T_{ij} 
\end{equation}

and

\begin{equation} \tag{12}
A_i = \frac{1}{\sum_j B_j D_j d_{ij}^{-\beta}}
\end{equation}

\begin{equation} \tag{13}
B_j = \frac{1}{\sum_i A_i O_i d_{ij}^{-\beta}}
\end{equation}

Now, the astute will have noticed that the calculation of $A_i$ relies on knowing $B_j$ and the calculation of $B_j$ relies on knowing $A_i$. A conundrum!! If I don’t know $A_i$ how can I calcuate $B_j$ and then in turn $A_i$ and then $B_j$ ad infinitum???!!

Well, I wrestled with that for a while until I came across [this paper by Martyn Senior](http://journals.sagepub.com/doi/abs/10.1177/030913257900300218) where he sketches out a very useful algorithm for iteratively arriving at values for $A_i$ and $B_j$ by setting each to equal to 1 initially and then continuing to calculate each in turn until the difference between each value is small enough not to matter.

We will return to this later, but for now, we will once again used the awesome power of Python to deal with all this difficulty for us!

We can run the doubly constrained model in exactly the same way as we ran the singly constrained models:

\begin{equation} \tag{14}
\lambda_{ij} = \exp (\alpha_i + \gamma_j -\beta \ln d_{ij})
\end{equation}

now in python:

## Model calibration

In [57]:
#create the formula (the "-1" indicates no intercept in the regression model).
#create the formula (the "-1" indicates no intercept in the regression model).
dbl_form = 'Total ~ Dest + Orig + log_Dist-1'
#run a doubly constrained sim
doubSim = smf.glm(formula = dbl_form, data=london_flows, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
#print(doubSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                61413
Model:                            GLM   Df Residuals:                    60617
Model Family:                 Poisson   Df Model:                          795
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.7074e+05
Date:                Mon, 01 May 2023   Deviance:                   1.7693e+06
Time:                        13:32:04   Pearson chi2:                 2.47e+06
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
De

$\beta$=0.0907

In [58]:
beta_pow=-doubSim.params[-1]
beta_pow

0.9096556995428264

In [59]:
#get the estimates
london_flows["doubsimfitted_pow"] = np.round(doubSim.mu)


In [60]:
pd.pivot_table(london_flows,values ="doubsimfitted_pow", index="Orig", columns = "Dest",
                            aggfunc=np.sum, margins=True)

Dest,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
Orig,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,600.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1224.0
Acton Town,NaN,NaN,NaN,15.0,15.0,NaN,11.0,1.0,NaN,17.0,...,30.0,3.0,5.0,12.0,NaN,2.0,NaN,2.0,NaN,3747.0
Aldgate,NaN,NaN,2.0,NaN,42.0,NaN,NaN,0.0,NaN,19.0,...,7.0,NaN,2.0,2.0,NaN,1.0,NaN,1.0,NaN,2873.0
Aldgate East,NaN,NaN,2.0,49.0,NaN,NaN,1.0,0.0,NaN,21.0,...,8.0,1.0,3.0,2.0,NaN,1.0,NaN,1.0,NaN,3172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,7.0,31.0,33.0,NaN,NaN,NaN,NaN,29.0,...,29.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,4862.0
Woodgrange Park,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530.0
Woodside Park,NaN,NaN,5.0,18.0,17.0,NaN,3.0,NaN,NaN,22.0,...,21.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,3093.0


In [61]:
# check goodness of fit
CalcRSqaured(london_flows['Total'],london_flows['doubsimfitted_pow'])

0.40772748658809344

With these parameters, the inverse power function has a far more rapid distance decay effect than the negative exponential function. In real life, what this means is that if the observed interactions drop off very rapidly with distance, then they might be more likely to follow an inverse power law. This might be the case when looking at trips to the local convenience store by walking, for example. On the other hand, if the effect of distance is less severe (for example migration across the country for a new job) then the negative exponential funtion might be more appropriate.

There is no hard and fast rule as to which function to pick, it will just come down to which fits the data better…

As [Taylor Oshan points out in his excellent Primer](http://openjournals.wu.ac.at/region/paper_175/175.html) what this means in our Poisson regression model is that we simply substitute $-\beta \ln d_{ij}$ for $-\beta d_{ij}$ in our model:

In [62]:
# Run a doubly constrained SIM with a negative exponential cost function.
doubsim_form = "Total ~ Orig + Dest + Dist -1"
doubsim_exp = smf.glm(formula=doubsim_form, data = london_flows, family = sm.families.Poisson()).fit()
#print(doubsim_exp.summary())

In [63]:
beta_exp=-doubsim_exp.params[-1]
beta_exp

0.00015436845765522456

In [64]:
# fit to make predictions
london_flows['doubsimfitted_exp']=np.round(doubsim_exp.mu,0)

In [65]:
pd.pivot_table(london_flows,values ="doubsimfitted_exp", index="Orig", columns = "Dest",
                            aggfunc=np.sum, margins=True)

Dest,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
Orig,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,603.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1221.0
Acton Town,NaN,NaN,NaN,11.0,10.0,NaN,17.0,0.0,NaN,12.0,...,40.0,4.0,2.0,19.0,NaN,0.0,NaN,1.0,NaN,3752.0
Aldgate,NaN,NaN,1.0,NaN,32.0,NaN,NaN,0.0,NaN,23.0,...,7.0,NaN,3.0,2.0,NaN,1.0,NaN,1.0,NaN,2883.0
Aldgate East,NaN,NaN,2.0,38.0,NaN,NaN,0.0,0.0,NaN,24.0,...,7.0,1.0,3.0,2.0,NaN,1.0,NaN,1.0,NaN,3167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,38.0,44.0,NaN,NaN,NaN,NaN,25.0,...,7.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,4862.0
Woodgrange Park,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,528.0
Woodside Park,NaN,NaN,2.0,17.0,15.0,NaN,0.0,NaN,NaN,25.0,...,10.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3093.0


In [66]:
# check the goodness-of-fit
CalcRSqaured(london_flows['Total'],london_flows['doubsimfitted_exp'])

0.4978623570468335

## Plug the parameter estimates back into Wilson's entropy maximising multiplicative models 
(To generate estimates)

the key to the doubly constrained models is the $A_i$ and $B_j$ balancing factors and as they rely on each other, they need to be calculated iteratively. We can do this using [Senior’s algorthim](http://journals.sagepub.com/doi/abs/10.1177/030913257900300218) also mentioned earlier.

Here is the code as provided by [Dan Lewis](https://github.com/danlewis85/UCL_CASA_Urban_Simulation/blob/master/Constrained%20SIM.ipynb) who in a departure from Dennet rewrites the algorithm as a function, which can then be called subject to the required parameters. In order for it to work it requires:

- pd - a pandas dataframe of origin-destination pairwise flows and associated data.
- orig_field - the name of the dataframe field in pd that uniquely labels origin zones.
- dest_field - the name of the dataframe field in pd that uniquely labels destination zones.
- Oi_field - the name of the dataframe field that stores total flows from a given origin $i$
- Dj_field - the name of the dataframe field that stores total flows to a given destination $j$
- cij_field - the name of the dataframe field that stores the pairwise cost (e.g. distance) between $i$ and $j$
- beta - a constant for the beta parameter you wish to use in the model
- cost_function - a string representing the cost function, either 'power' or 'exponential'
- Ainame - What you want to call the new field in pd that will hold $A_{i}$ values, defaults to "Ai_new"
- Bjname - What you want to call the new field in pd that will hold $B_{j}$ values, defaults to "Bj_new"
- converge - A threshold value at which a model can be said to have converged, the default of 0.001 seems to work fine.

In [67]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take london_flows ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(london_flows.groupby(["Orig"])["Total"].agg(np.sum))
O_i.rename(columns={"Total":"O_i"}, inplace = True)
london_flows = london_flows.merge(O_i, on = "Orig", how = "left" )

D_j = pd.DataFrame(london_flows.groupby(["Dest"])["Total"].agg(np.sum))
D_j.rename(columns={"Total":"D_j"}, inplace = True)
london_flows = london_flows.merge(D_j, on = "Dest", how = "left" )

In [69]:

# Here is the entropy maximising approach for a known beta.
# Plug in the required values in this function to solve.

def balance_doubly_constrained(pd, orig_field, dest_field, Oi_field, Dj_field, cij_field, beta, 
                               cost_function, Ai_name = "Ai_new", Bj_name = "Bj_new", converge=0.001):
    # Define some variables
    Oi = pd[[orig_field, Oi_field]]
    Dj = pd[[dest_field,Dj_field]]    
    if cost_function.lower() in ['power','pow']:
        beta_cij = np.exp(beta * np.log(pd[cij_field]))
    elif cost_function.lower() in ['exponential','exp']:
        beta_cij = np.exp(beta * pd[cij_field])
    else:
        return "Cost function not specified properly, use 'exp' or 'pow'"
    
    # Create some helper variables
    cnvg = 1
    iteration = 0
    # Now iteratively rebalance the Ai and Bj terms until convergence
    while cnvg > converge:
        if iteration == 0:
            # This first condition sets starting values for Ai and Bj
            # NB sets starting value of Ai assuming Bj is a vector of 1s.
            # We've already established beta_cij with the appropriate cost function, so...
            Oi = Oi.assign(Ai = Dj[Dj_field] * beta_cij)
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Now set up Bjs using starting values of Ai
            Dj = Dj.assign(Bj = Oi['Ai_old'] * Oi[Oi_field] * beta_cij)
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True, suffixes = ('','_old'))
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Increment loop
            iteration += 1
        else:
            # This bit is the iterated bit of the loop which refines the values of Ai and Bj
            # First Ai
            Oi['Ai'] = Dj['Bj_old'] * Dj[Dj_field] * beta_cij
            # Aggregate Ai and take inverse
            Ai = 1.0/Oi.groupby(orig_field)['Ai'].sum().to_frame()
            # Drop temporary Ai
            Oi.drop('Ai', axis=1, inplace=True)
            # Merge new Ais 
            Oi = Oi.merge(Ai,left_on = orig_field, right_index = True)
            # Calculate the difference between old and new Ais
            Oi['diff'] = np.absolute((Oi['Ai_old'] - Oi['Ai'])/Oi['Ai_old'])
            # Set new Ais to Ai_old
            Oi['Ai_old'] = Oi['Ai']
            # Drop the temporary Ai field we created, leaving Ai_old
            Oi.drop('Ai', axis=1, inplace=True)
            
            # Then Bj
            Dj['Bj'] = Oi['Ai_old'] * Oi[Oi_field] * beta_cij
            # Aggregate Bj and take inverse
            Bj = 1.0/Dj.groupby(dest_field)['Bj'].sum().to_frame()
            # Drop temporary Bj
            Dj.drop('Bj', axis=1, inplace=True)
            # Merge new Bjs
            Dj = Dj.merge(Bj,left_on = dest_field, right_index = True)
            # Calculate the difference between old and new Bjs
            Dj['diff'] = np.absolute((Dj['Bj_old'] - Dj['Bj'])/Dj['Bj_old'])
            # Set new Bjs to Bj_old
            Dj['Bj_old'] = Dj['Bj']
            # Drop the temporary Bj field we created, leaving Bj_old
            Dj.drop('Bj', axis=1, inplace=True)
            
            # Assign higher sum difference from Ai or Bj to cnvg
            cnvg = np.maximum(Oi['diff'].sum(),Dj['diff'].sum())
            
            # Print and increment loop
            print("Iteration:", iteration)
            iteration += 1

    # When the while loop finishes add the computed Ai_old and Bj_old to the dataframe and return
    pd[Ai_name] = Oi['Ai_old']
    pd[Bj_name] = Dj['Bj_old']
    return pd

Use the maximise entropy function to calculate $A_1$ and $B_j$ for the Poisson model 
 by plugging the beta

In [70]:
# Inver power cost function
# Get the balancing factors.
london_flows = balance_doubly_constrained(london_flows,'Orig','Dest','O_i','D_j','Dist',-beta_pow,'power')

# Now predict the model again using the new Ai and Dj fields.
london_flows['SIM_est_pow'] = np.round(london_flows['O_i'] * london_flows['Ai_new'] * london_flows['D_j'] * london_flows['Bj_new'] * 
                                   np.exp(np.log(london_flows['Dist'])*-beta_pow))


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22


In [71]:

# Check out the matrix
pd.pivot_table(london_flows,values='SIM_est_pow',index ='Orig',columns='Dest',fill_value=0,aggfunc=sum,margins=True)

Dest,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
Orig,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,26,600.0
Acton Central,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,1224.0
Acton Town,0,0,0,15,15,0,11,1,0,17,...,30,3,5,12,0,2,0,2,0,3747.0
Aldgate,0,0,2,0,42,0,0,0,0,19,...,7,0,2,2,0,1,0,1,0,2873.0
Aldgate East,0,0,2,49,0,0,1,0,0,21,...,8,1,3,2,0,1,0,1,0,3172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0,0,7,31,33,0,0,0,0,29,...,29,0,10,0,0,0,0,0,0,4862.0
Woodgrange Park,0,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,530.0
Woodside Park,0,0,5,18,17,0,3,0,0,22,...,21,0,6,0,0,0,0,0,0,3093.0


In [ ]:
# check the goodness-of-fit
CalcRSqaured (london_flows['Total'],london_flows['SIM_est_pow'])

### Exponential cost function

In [ ]:
# Get the balancing factors. NB Setting of new field names for Ai and Bj.
london_flows = balance_doubly_constrained(london_flows,'Orig','Dest','O_i','D_j','Dist',-beta_exp,'exponential','Ai_exp','Bj_exp')

# Now predict the model again using the new Ai and Dj fields.
london_flows['SIM_est_exp'] = np.round(london_flows['O_i'] * london_flows['Ai_exp'] * london_flows['D_j'] * london_flows['Bj_exp'] * 
                                   np.exp(london_flows['Dist']*-beta_exp))
# Check out the matrix
pd.pivot_table(london_flows,values='SIM_est_exp',index ='Orig',columns='Dest',fill_value=0,aggfunc=sum,margins=True)

In [ ]:
# check the goodness-of-fit
CalcRSqaured (london_flows['Total'],london_flows['SIM_est_exp'])

# Scenario

## Scenario A:

 assume that **Canary Wharf** has a 50% decrease in jobs after Brexit. Using the 
calibrated parameter $\beta$, compute the new flows for scenario A. Make sure the number of commuters is conserved, and explain how you ensured this.


In [ ]:
# Create a new variable with the altered jobs
def new_job(row):
    if row["Dest"] == "Canary Wharf":
        val = 0.5*row["Dj_destjob"]
    else:
        val = row["Dj_destjob"]
    return val
        
london_flows["Dj_destjob_a"] = london_flows.apply(new_job, axis =1)
#london_flows.head(10)

In [ ]:
# Get the balancing factors. NB Setting of new field names for Ai and Bj.
london_flows = balance_doubly_constrained(london_flows,'Orig','Dest','O_i',"Dj_destjob_a",'Dist',-beta_exp,'exponential','Ai_exp_a','Bj_exp_a')

# Now predict the model again using the new Ai and Dj fields.
london_flows['SIM_est_exp_a'] = np.round(london_flows['O_i'] * london_flows['Ai_exp_a'] * london_flows['Dj_destjob_a'] * london_flows['Bj_exp_a'] * 
                                   np.exp(london_flows['Dist']*-beta_exp))
# Check out the matrix
pd.pivot_table(london_flows,values='SIM_est_exp',index ='Orig',columns='Dest',fill_value=0,aggfunc=sum,margins=True)

# Scenario B

set up the adjusted $\beta$, we assume that $k$ is the increase in cost of transport, meaning the original cost is $c$ and new cost is $kc$

According to research (nature of transport cost and pricing), the  transport costs are proportional to distance
$f(d)=kd_{i j}$
Given a cost function that aligns with power law
$f(d)=d_{i j}^{-\beta}$
To adjust the parameter $\beta$ in the cost function reflecting the increase in cost of transport

$(kd_{i j})^{-\beta}=d_{i j}^{-\beta_{adjusted}}$

After taking logarithmic of both sides, the equation will look like the following 

$ \beta(log(k)+ log(d_{i j})=\beta_{adjusted}log(d_{ij})$


So $\beta_{adjusted}=\beta \frac{ log(d_{i j})+log(k)}{log(d_{i j})}$

In [ ]:
def beta_b_pow(beta,k):
    #beta: the calibrated beta
    beta_adj=beta*(london_flows['log_Dist']+np.log(k))/london_flows['log_Dist']
    return beta_adj.mean()

In [ ]:
# assume the cost of transport has increased 25%
beta_pow_25=beta_b(beta_pow,1.25)
beta_pow_25

In [ ]:
# Inver power cost function
# Get the balancing factors.
london_flows_b = balance_doubly_constrained(london_flows.copy(),'Orig','Dest','O_i','D_j','Dist',-beta_pow_25,'power')

# Now predict the model again using the new Ai and Dj fields.
london_flows_b['SIM_est_pow_25'] = np.round(london_flows['O_i'] * london_flows['Ai_new'] * london_flows['D_j'] * london_flows['Bj_new'] * 
                                   np.exp(np.log(london_flows['Dist'])*-beta_pow_25))


In [ ]:

# Check out the matrix
pd.pivot_table(london_flows_b,values='SIM_est_pow_25',index ='Orig',columns='Dest',fill_value=0,aggfunc=sum,margins=True)

In [ ]:
# assume the cost of transport has increased 50%
beta_exp_50=beta_b(beta_exp,1.5)
beta_exp_50

In [ ]:
# Get the balancing factors. NB Setting of new field names for Ai and Bj.
london_flows_b = balance_doubly_constrained(london_flows,'Orig','Dest','O_i','D_j','Dist',-beta_exp_50,'exponential','Ai_exp','Bj_exp')

# Now predict the model again using the new Ai and Dj fields.
london_flows_b['SIM_est_exp_b2'] = np.round(london_flows['O_i'] * london_flows['Ai_exp'] * london_flows['D_j'] * london_flows['Bj_exp'] * 
                                   np.exp(london_flows['Dist']*-beta_exp_50))
# Check out the matrix
pd.pivot_table(london_flows,values='SIM_est_exp_b2',index ='Orig',columns='Dest',fill_value=0,aggfunc=sum,margins=True)